In [9]:
import sys
relative_path = "../../"
sys.path.append(relative_path)
import os
from lookup import LookupCreator
from sensepolar.polarity import WordPolarity
from sensepolar.embed.bertEmbed import BERTWordEmbeddings
from sensepolar.embed.albertEmbed import ALBERTWordEmbeddings
from sensepolar.embed.robertaEmbed import RoBERTaWordEmbeddings
from sensepolar.polarDim import PolarDimensions
from sensepolar.oracle.dictionaryapi import Dictionary
import nltk
import pandas as pd
from nltk.stem import PorterStemmer
from datasets import load_dataset
from datasets import Dataset as Data
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from transformers import BertTokenizer, BertModel
from IPython.utils import io
import re
import torch.nn.utils.rnn as rnn_utils

In [10]:
class PoemSentimentDataset(Dataset):
    def __init__(self, verse_text, labels, word_polarity_model, method='cls', dimension=39):
        self.verse_text = verse_text
        self.labels = labels
        self.word_polarity_model = word_polarity_model
        self.polar_embeddings_cache = {}
        self.dimension = dimension
        self.method = method

    def __len__(self):
        return len(self.verse_text)

    def get_sense_polar_embedding(self, word, context):
        if (word, context) not in self.polar_embeddings_cache:
            with io.capture_output() as captured:
                polar_embedding = self.word_polarity_model.analyze_word(word, context)
            antonym_dict = {}
            for pair in polar_embedding:
                antonym_dict[(pair[0], pair[1])] = antonym_dict.get((pair[0], pair[1]), []) + [pair[2]]
            sorted_antonym_dict = dict(sorted(antonym_dict.items(), key=lambda item: item[0]))
            self.polar_embeddings_cache[(word, context)] = list(sorted_antonym_dict.values())
        return self.polar_embeddings_cache[(word, context)]

    def __getitem__(self, idx):
        verse = self.verse_text[idx]
        labels = self.labels[idx]
        verse_polar_embeddings = None

        if self.method == 'cls':
            verse += ' [CLS]'
            cls_polar_embedding = self.get_sense_polar_embedding('[CLS]', verse)
            verse_polar_embeddings = torch.tensor(cls_polar_embedding, dtype=torch.float)
        else:
            polar_embeddings_list = []
            for word in verse.split():
                polar_embedding = self.get_sense_polar_embedding(word, verse)
                polar_embeddings_list.append(polar_embedding)
            verse_polar_embeddings = torch.tensor(polar_embeddings_list, dtype=torch.float)
            verse_polar_embeddings = torch.mean(verse_polar_embeddings, dim=0)
            
        verse_polar_embeddings = verse_polar_embeddings.long()
        verse_polar_embeddings = verse_polar_embeddings.squeeze(dim=1)
        verse_polar_embeddings = verse_polar_embeddings[:self.dimension]
        label = torch.tensor(labels, dtype=torch.long)

        return {
            'polar_embeddings': verse_polar_embeddings,
            'label': label
        }


In [11]:

class PolarEmbeddingClassifier(nn.Module):
    def __init__(self, num_classes, polar_dimension, model_name='sense_polar_model'):
        super(PolarEmbeddingClassifier, self).__init__()
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(polar_dimension, num_classes)
        self.model_name = model_name

    def forward(self, polar_embeddings):
        output = self.dropout(polar_embeddings.float())
        logits = self.fc(output)
        return logits

    def train_model(self, train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device):
        best_valid_loss = float('inf')
        epochs_without_improvement = 0
        train_losses = []
        valid_losses = [] 

        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0
            for batch in train_loader:
                polar_embeddings = batch['polar_embeddings'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()

                outputs = self(polar_embeddings)  
                logits = outputs
                loss = loss_fn(logits, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            avg_train_loss = train_loss / len(train_loader)
            train_losses.append(avg_train_loss)
            print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss}")

            # Validation
            self.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for batch in valid_loader:
                    polar_embeddings = batch['polar_embeddings'].to(device)  
                    labels = batch['label'].to(device)

                    outputs = self(polar_embeddings)
                    logits = outputs
                    loss = loss_fn(logits, labels)

                    valid_loss += loss.item()

            avg_valid_loss = valid_loss / len(valid_loader)
            valid_losses.append(avg_valid_loss)
            print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {avg_valid_loss}")

            if avg_valid_loss < best_valid_loss:
                best_valid_loss = avg_valid_loss
                epochs_without_improvement = 0
                torch.save(self.state_dict(), "model/" + self.model_name + ".pth")
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print(f"Early stopping. No improvement in {patience} epochs.")
                    break

        self.load_state_dict(torch.load("model/" + self.model_name +".pth"))
        # Save results in a file
        with open('results/' + self.model_name+'_train.txt', 'w') as f:
            for epoch, train_loss, valid_loss in zip(range(1, num_epochs + 1), train_losses, valid_losses):
                f.write(f"Epoch {epoch} - Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}\n")


    def test_model(self, test_loader, loss_fn, device):
        self.eval()
        test_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        all_predictions = []  # Initialize list to collect predictions
        all_labels = [] 

        with torch.no_grad():
            for batch in test_loader:
                polar_embeddings = batch['polar_embeddings'].to(device) 
                labels = batch['label'].to(device)

                outputs = self(polar_embeddings) 
                logits = outputs
                loss = loss_fn(logits, labels)
                test_loss += loss.item()

                _, predicted = torch.max(logits, dim=1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)
                all_predictions.extend(predicted.tolist())
                all_labels.extend(labels.tolist())

        avg_test_loss = test_loss / len(test_loader)
        accuracy = correct_predictions / total_samples
        print(f"Test Loss: {avg_test_loss}, Accuracy: {accuracy}")
        classification_rep = classification_report(all_labels, all_predictions, digits=4)
        with open('results/' + self.model_name+'_test.txt', 'w') as f:
            f.write(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}\n")
            f.write("\nClassification Report:\n")
            f.write(classification_rep)

    def predict(self, polar_embeddings, device): 
        polar_embeddings = polar_embeddings.to(device)
        
        self.eval()
        with torch.no_grad():
            outputs = self(polar_embeddings)
            logits = outputs

        _, predicted = torch.max(logits, dim=1)
        return predicted.item()


In [15]:
from itertools import product
from sklearn.metrics import classification_report

experiment_settings = {
        "embed_model": [ALBERTWordEmbeddings],  
        "polar_dimension": [ 10], 
        "WordPolarity_method": ["base-change", "projection"], 
        "PoemSentimentDataset_method": ["avg", "cls"], 
        "layer": [2, 3, 4, 5]  
    }
# Create a list of lists containing values for each setting
setting_values = [values for values in experiment_settings.values()]

# Iterate through all combinations of experiment settings
for setting_combination in product(*setting_values):
    setting = {
        key: value for key, value in zip(experiment_settings.keys(), setting_combination)
    }
    print(setting)
    with io.capture_output() as captured:
        # Extract the values from the current setting
        embed_model = setting["embed_model"]
        polar_dimension = setting["polar_dimension"]
        WordPolarity_method = setting["WordPolarity_method"]
        PoemSentimentDataset_method = setting["PoemSentimentDataset_method"]
        layer = setting["layer"]
        
        print('Setting', embed_model, polar_dimension, WordPolarity_method, PoemSentimentDataset_method, layer)
        dataset = load_dataset("poem_sentiment")
        out_path = './antonyms/'
        antonym_path = "data/polars_all_combined.xlsx"
        embed_model = embed_model(layer=layer)
        
        dictionary = Dictionary('wordnet', api_key='')    
        lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path, is_path=True)
        lookupSpace.create_lookup_files()
        antonym_path = out_path + "polar_dimensions.pkl"

        pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
        pdc.create_polar_dimensions(out_path,"/polar_dimensions.pkl" )

        wp = WordPolarity(embed_model, antonym_path=antonym_path, method=WordPolarity_method)
        num_classes = 4

    # Define your model
    sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=polar_dimension, model_name=f'sense_polar_{embed_model.model_name}_dim{polar_dimension}_{WordPolarity_method}_{PoemSentimentDataset_method}_layer{layer}')


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sensepolar_model.to(device)

    optimizer = torch.optim.AdamW(sensepolar_model.parameters(), lr=1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()

    num_epochs = 1000
    patience = 50

    preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
    train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
    test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
    valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
    train_labels = dataset["train"]["label"]
    test_labels = dataset["test"]["label"]
    valid_labels = dataset["validation"]["label"]

    # train_texts_filtered = []
    # train_labels_filtered = []
    # for text, label in zip(train_texts, train_labels):
    #     if label != 3:
    #         train_texts_filtered.append(text)
    #         train_labels_filtered.append(label)`

    train_dataset = PoemSentimentDataset(train_texts, train_labels, wp, method=PoemSentimentDataset_method, dimension=polar_dimension)
    valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=polar_dimension, method=PoemSentimentDataset_method)
    test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=polar_dimension, method=PoemSentimentDataset_method)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

    sensepolar_model.test_model(test_loader, loss_fn, device)

{'embed_model': <class 'sensepolar.embed.albertEmbed.ALBERTWordEmbeddings'>, 'polar_dimension': 10, 'WordPolarity_method': 'base-change', 'PoemSentimentDataset_method': 'avg', 'layer': 2}


Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 2.2934774977207992e+18
Epoch 1/1000 - Validation Loss: 1.0702493431191868e+18
Epoch 2/1000 - Training Loss: 1.359855860845492e+18
Epoch 2/1000 - Validation Loss: 2.397095010893075e+17
Epoch 3/1000 - Training Loss: 1.0088591513779662e+18
Epoch 3/1000 - Validation Loss: 1.95768856460769e+17
Epoch 4/1000 - Training Loss: 1.0200755186010124e+18
Epoch 4/1000 - Validation Loss: 2.890717136139247e+17
Epoch 5/1000 - Training Loss: 8.759815537992663e+17
Epoch 5/1000 - Validation Loss: 1.461137676619706e+17
Epoch 6/1000 - Training Loss: 7.963574692449016e+17
Epoch 6/1000 - Validation Loss: 1.903411013575171e+17
Epoch 7/1000 - Training Loss: 6.807617651906595e+17
Epoch 7/1000 - Validation Loss: 1.1147599888103131e+17
Epoch 8/1000 - Training Loss: 6.938457763937929e+17
Epoch 8/1000 - Validation Loss: 2.0879510572248035e+17
Epoch 9/1000 - Training Loss: 6.125154479751823e+17
Epoch 9/1000 - Validation Loss: 1.449287984563393e+17
Epoch 10/1000 - Training Loss: 4.90090952

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

{'embed_model': <class 'sensepolar.embed.albertEmbed.ALBERTWordEmbeddings'>, 'polar_dimension': 10, 'WordPolarity_method': 'base-change', 'PoemSentimentDataset_method': 'avg', 'layer': 3}


Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 2.761709329864046e+18
Epoch 1/1000 - Validation Loss: 1.5784540432035817e+18
Epoch 2/1000 - Training Loss: 1.882211883930803e+18
Epoch 2/1000 - Validation Loss: 7.595994880175917e+17
Epoch 3/1000 - Training Loss: 1.4103476365772718e+18
Epoch 3/1000 - Validation Loss: 5.965114954328872e+17
Epoch 4/1000 - Training Loss: 1.3094386831735427e+18
Epoch 4/1000 - Validation Loss: 5.343906837371102e+17
Epoch 5/1000 - Training Loss: 1.2170381759473068e+18
Epoch 5/1000 - Validation Loss: 5.90238030590715e+17
Epoch 6/1000 - Training Loss: 1.0375266928927328e+18
Epoch 6/1000 - Validation Loss: 4.723342769166371e+17
Epoch 7/1000 - Training Loss: 9.937621799644692e+17
Epoch 7/1000 - Validation Loss: 4.6422661315810336e+17
Epoch 8/1000 - Training Loss: 8.617744345714244e+17
Epoch 8/1000 - Validation Loss: 3.749635372054891e+17
Epoch 9/1000 - Training Loss: 7.754728441028842e+17
Epoch 9/1000 - Validation Loss: 3.828144060965285e+17
Epoch 10/1000 - Training Loss: 6.84051407

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

KeyboardInterrupt: 

## Roberta Runns

In [ ]:
from itertools import product
from sklearn.metrics import classification_report

experiment_settings = {
        "embed_model": [RoBERTaWordEmbeddings, BERTWordEmbeddings, ALBERTWordEmbeddings],  
        "polar_dimension": [ 786, 1586], 
        "WordPolarity_method": ["base-change", "projection"], 
        "PoemSentimentDataset_method": ["avg", "cls"], 
        "layer": [2, 3, 4, 5]  
    }
# Create a list of lists containing values for each setting
setting_values = [values for values in experiment_settings.values()]

# Iterate through all combinations of experiment settings
for setting_combination in product(*setting_values):
    setting = {
        key: value for key, value in zip(experiment_settings.keys(), setting_combination)
    }
    print(setting)
    with io.capture_output() as captured:
        # Extract the values from the current setting
        embed_model = setting["embed_model"]
        polar_dimension = setting["polar_dimension"]
        WordPolarity_method = setting["WordPolarity_method"]
        PoemSentimentDataset_method = setting["PoemSentimentDataset_method"]
        layer = setting["layer"]
        
        print('Setting', embed_model, polar_dimension, WordPolarity_method, PoemSentimentDataset_method, layer)
        dataset = load_dataset("poem_sentiment")
        out_path = './antonyms/'
        antonym_path = "data/polars_all_combined.xlsx"
        embed_model = embed_model(layer=layer)
        
        dictionary = Dictionary('wordnet', api_key='')    
        lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
        lookupSpace.create_lookup_files()
        antonym_path = out_path + "polar_dimensions.pkl"

        pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
        pdc.create_polar_dimensions(out_path,"/polar_dimensions.pkl" )

        wp = WordPolarity(embed_model, antonym_path=antonym_path, method=WordPolarity_method)
        num_classes = 4

    # Define your model
    sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=polar_dimension, model_name=f'sense_polar_{embed_model.model_name}_dim{polar_dimension}_{WordPolarity_method}_{PoemSentimentDataset_method}_layer{layer}')


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sensepolar_model.to(device)

    optimizer = torch.optim.AdamW(sensepolar_model.parameters(), lr=1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()

    num_epochs = 1000
    patience = 50

    preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
    train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
    test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
    valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
    train_labels = dataset["train"]["label"]
    test_labels = dataset["test"]["label"]
    valid_labels = dataset["validation"]["label"]

    # train_texts_filtered = []
    # train_labels_filtered = []
    # for text, label in zip(train_texts, train_labels):
    #     if label != 3:
    #         train_texts_filtered.append(text)
    #         train_labels_filtered.append(label)`

    train_dataset = PoemSentimentDataset(train_texts, train_labels, wp, method=PoemSentimentDataset_method, dimension=polar_dimension)
    valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=polar_dimension, method=PoemSentimentDataset_method)
    test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=polar_dimension, method=PoemSentimentDataset_method)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

    sensepolar_model.test_model(test_loader, loss_fn, device)

{'embed_model': <class 'sensepolar.embed.robertaEmbed.RoBERTaWordEmbeddings'>, 'polar_dimension': 786, 'WordPolarity_method': 'base-change', 'PoemSentimentDataset_method': 'avg', 'layer': 2}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 1.3567408961909158
Epoch 1/1000 - Validation Loss: 1.3329007455280848
Epoch 2/1000 - Training Loss: 1.3236245619399207
Epoch 2/1000 - Validation Loss: 1.2962783915655953
Epoch 3/1000 - Training Loss: 1.292531070964677
Epoch 3/1000 - Validation Loss: 1.2629247052328927
Epoch 4/1000 - Training Loss: 1.263562730380467
Epoch 4/1000 - Validation Loss: 1.2321505546569824
Epoch 5/1000 - Training Loss: 1.2383650988340378
Epoch 5/1000 - Validation Loss: 1.2041877167565482
Epoch 6/1000 - Training Loss: 1.2154263917888914
Epoch 6/1000 - Validation Loss: 1.1778766768319267
Epoch 7/1000 - Training Loss: 1.1941535025835037
Epoch 7/1000 - Validation Loss: 1.1551572935921806
Epoch 8/1000 - Training Loss: 1.1751979451094354
Epoch 8/1000 - Validation Loss: 1.134286301476615
Epoch 9/1000 - Training Loss: 1.1585263652460915
Epoch 9/1000 - Validation Loss: 1.1156844411577498
Epoch 10/1000 - Training Loss: 1.143482655286789
Epoch 10/1000 - Validation Loss: 1.0989598802157812
Ep

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 1/1000 - Training Loss: 1.3638626549925124
Epoch 1/1000 - Validation Loss: 1.3418146882738387
Epoch 2/1000 - Training Loss: 1.3297476917505264
Epoch 2/1000 - Validation Loss: 1.3045054503849574
Epoch 3/1000 - Training Loss: 1.2981198813234056
Epoch 3/1000 - Validation Loss: 1.2705982412610735
Epoch 4/1000 - Training Loss: 1.269454572881971
Epoch 4/1000 - Validation Loss: 1.2396946123668127
Epoch 5/1000 - Training Loss: 1.243673371417182
Epoch 5/1000 - Validation Loss: 1.2112106084823608
Epoch 6/1000 - Training Loss: 1.2190798159156526
Epoch 6/1000 - Validation Loss: 1.1857751096997942
Epoch 7/1000 - Training Loss: 1.1982983989374978
Epoch 7/1000 - Validation Loss: 1.1622588804789953
Epoch 8/1000 - Training Loss: 1.1793300647820746
Epoch 8/1000 - Validation Loss: 1.1407230922154017
Epoch 9/1000 - Training Loss: 1.1622239585433687
Epoch 9/1000 - Validation Loss: 1.1218615514891488
Epoch 10/1000 - Training Loss: 1.1475104444793292
Epoch 10/1000 - Validation Loss: 1.1049675260271346


...

Epoch 1/1000 - Training Loss: 1.367193334868976
Epoch 1/1000 - Validation Loss: 1.3451264245169503
Epoch 2/1000 - Training Loss: 1.332215170775141
Epoch 2/1000 - Validation Loss: 1.3079540048326765
Epoch 3/1000 - Training Loss: 1.3005724868604116
Epoch 3/1000 - Validation Loss: 1.273998430797032
Epoch 4/1000 - Training Loss: 1.271752574614116
Epoch 4/1000 - Validation Loss: 1.241897668157305
Epoch 5/1000 - Training Loss: 1.2452910074165888
Epoch 5/1000 - Validation Loss: 1.2137842178344727
Epoch 6/1000 - Training Loss: 1.2221065951245171
Epoch 6/1000 - Validation Loss: 1.1870623145784651
Epoch 7/1000 - Training Loss: 1.1999733043568475
Epoch 7/1000 - Validation Loss: 1.1640618188040597
Epoch 8/1000 - Training Loss: 1.1814669136490141
Epoch 8/1000 - Validation Loss: 1.1425334044865199
Epoch 9/1000 - Training Loss: 1.1639749705791473
Epoch 9/1000 - Validation Loss: 1.123564030442919
Epoch 10/1000 - Training Loss: 1.1487741385187422
Epoch 10/1000 - Validation Loss: 1.1061015639986311
Epoc

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 1/1000 - Training Loss: 1.367909461259842
Epoch 1/1000 - Validation Loss: 1.3468393768583025
Epoch 2/1000 - Training Loss: 1.3333286323717661
Epoch 2/1000 - Validation Loss: 1.3102714163916451
Epoch 3/1000 - Training Loss: 1.3014954520123345
Epoch 3/1000 - Validation Loss: 1.2758963789258684
Epoch 4/1000 - Training Loss: 1.2733687588146754
Epoch 4/1000 - Validation Loss: 1.2444442680903844
Epoch 5/1000 - Training Loss: 1.2466811324868883
Epoch 5/1000 - Validation Loss: 1.2166331154959542
Epoch 6/1000 - Training Loss: 1.2231871911457606
Epoch 6/1000 - Validation Loss: 1.190434353692191
Epoch 7/1000 - Training Loss: 1.202137759753636
Epoch 7/1000 - Validation Loss: 1.1667632375444685
Epoch 8/1000 - Training Loss: 1.1831863863127572
Epoch 8/1000 - Validation Loss: 1.1453818423407418
Epoch 9/1000 - Training Loss: 1.1654801272920199
Epoch 9/1000 - Validation Loss: 1.1262806398527963
Epoch 10/1000 - Training Loss: 1.1507030172007424
Epoch 10/1000 - Validation Loss: 1.1090389319828577
E

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 1/1000 - Training Loss: 1.3542797714471817
Epoch 1/1000 - Validation Loss: 1.3304969412939889
Epoch 2/1000 - Training Loss: 1.3200593463012151
Epoch 2/1000 - Validation Loss: 1.294840931892395
Epoch 3/1000 - Training Loss: 1.289409081850733
Epoch 3/1000 - Validation Loss: 1.2609183277402605
Epoch 4/1000 - Training Loss: 1.2616502791643143
Epoch 4/1000 - Validation Loss: 1.2297469547816686
Epoch 5/1000 - Training Loss: 1.2358872592449188
Epoch 5/1000 - Validation Loss: 1.2026441267558508
Epoch 6/1000 - Training Loss: 1.212911810193743
Epoch 6/1000 - Validation Loss: 1.1775644506726946
Epoch 7/1000 - Training Loss: 1.1923219689301081
Epoch 7/1000 - Validation Loss: 1.1547014032091414
Epoch 8/1000 - Training Loss: 1.1737427860498428
Epoch 8/1000 - Validation Loss: 1.1339802145957947
Epoch 9/1000 - Training Loss: 1.1587145839418684
Epoch 9/1000 - Validation Loss: 1.115404256752559
Epoch 10/1000 - Training Loss: 1.143590013895716
Epoch 10/1000 - Validation Loss: 1.0993403792381287
Epo

## Bert Runs

In [8]:
from itertools import product
from sklearn.metrics import classification_report

experiment_settings = {
        "embed_model": [RoBERTaWordEmbeddings, BERTWordEmbeddings, ALBERTWordEmbeddings],  
        "polar_dimension": [ 786, 1586], 
        "WordPolarity_method": ["base-change", "projection"], 
        "PoemSentimentDataset_method": ["avg", "cls"], 
        "layer": [2, 3, 4, 5], 
        "avg_embed": [True]
    }
# Create a list of lists containing values for each setting
setting_values = [values for values in experiment_settings.values()]

# Iterate through all combinations of experiment settings
for setting_combination in product(*setting_values):
    setting = {
        key: value for key, value in zip(experiment_settings.keys(), setting_combination)
    }
    print(setting)
    with io.capture_output() as captured:
        # Extract the values from the current setting
        embed_model = setting["embed_model"]
        polar_dimension = setting["polar_dimension"]
        WordPolarity_method = setting["WordPolarity_method"]
        PoemSentimentDataset_method = setting["PoemSentimentDataset_method"]
        layer = setting["layer"]
        avg_embed = setting["avg_embed"]
        
        print('Setting', embed_model, polar_dimension, WordPolarity_method, PoemSentimentDataset_method, layer)
        dataset = load_dataset("poem_sentiment")
        out_path = './antonyms/'
        antonym_path = "data/polars_all_combined.xlsx"
        embed_model = embed_model(layer=layer, avg_layers=avg_embed)
        
        dictionary = Dictionary('wordnet', api_key='')    
        lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
        lookupSpace.create_lookup_files()
        antonym_path = out_path + "polar_dimensions.pkl"

        pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
        pdc.create_polar_dimensions(out_path,"/polar_dimensions.pkl" )

        wp = WordPolarity(embed_model, antonym_path=antonym_path, method=WordPolarity_method)
        num_classes = 4

    # Define your model
    sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=polar_dimension, model_name=f'sense_polar_{embed_model.model_name}_dim{polar_dimension}_{WordPolarity_method}_{PoemSentimentDataset_method}_layer{layer}')


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sensepolar_model.to(device)

    optimizer = torch.optim.AdamW(sensepolar_model.parameters(), lr=1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()

    num_epochs = 1000
    patience = 50

    preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
    train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
    test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
    valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
    train_labels = dataset["train"]["label"]
    test_labels = dataset["test"]["label"]
    valid_labels = dataset["validation"]["label"]

    # train_texts_filtered = []
    # train_labels_filtered = []
    # for text, label in zip(train_texts, train_labels):
    #     if label != 3:
    #         train_texts_filtered.append(text)
    #         train_labels_filtered.append(label)`

    train_dataset = PoemSentimentDataset(train_texts, train_labels, wp, method=PoemSentimentDataset_method, dimension=polar_dimension)
    valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=polar_dimension, method=PoemSentimentDataset_method)
    test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=polar_dimension, method=PoemSentimentDataset_method)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

    sensepolar_model.test_model(test_loader, loss_fn, device)

{'embed_model': <class 'sensepolar.embed.bertEmbed.BERTWordEmbeddings'>, 'polar_dimension': 786, 'WordPolarity_method': 'base-change', 'PoemSentimentDataset_method': 'avg', 'layer': 2}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 1.3687065882342202
Epoch 1/1000 - Validation Loss: 1.346116542816162
Epoch 2/1000 - Training Loss: 1.3339994528463908
Epoch 2/1000 - Validation Loss: 1.3089594841003418
Epoch 3/1000 - Training Loss: 1.30201789523874
Epoch 3/1000 - Validation Loss: 1.2746562106268746
Epoch 4/1000 - Training Loss: 1.2721277879817146
Epoch 4/1000 - Validation Loss: 1.2432794741221838
Epoch 5/1000 - Training Loss: 1.2460773821387972
Epoch 5/1000 - Validation Loss: 1.2142359529222762
Epoch 6/1000 - Training Loss: 1.221897570150239
Epoch 6/1000 - Validation Loss: 1.187694753919329
Epoch 7/1000 - Training Loss: 1.2009116794381822
Epoch 7/1000 - Validation Loss: 1.1636891875948225
Epoch 8/1000 - Training Loss: 1.1808439024857111
Epoch 8/1000 - Validation Loss: 1.1427658115114485
Epoch 9/1000 - Training Loss: 1.1638422438076563
Epoch 9/1000 - Validation Loss: 1.123573797089713
Epoch 10/1000 - Training Loss: 1.1499734286751067
Epoch 10/1000 - Validation Loss: 1.1060173681804113
Epoc

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 1/1000 - Training Loss: 1.3711503722837992
Epoch 1/1000 - Validation Loss: 1.3481003727231706
Epoch 2/1000 - Training Loss: 1.3358537554740906
Epoch 2/1000 - Validation Loss: 1.310742906161717
Epoch 3/1000 - Training Loss: 1.3033654178891863
Epoch 3/1000 - Validation Loss: 1.2763084173202515
Epoch 4/1000 - Training Loss: 1.2742787897586823
Epoch 4/1000 - Validation Loss: 1.244286230632237
Epoch 5/1000 - Training Loss: 1.2470363633973258
Epoch 5/1000 - Validation Loss: 1.215685316494533
Epoch 6/1000 - Training Loss: 1.2231273182800837
Epoch 6/1000 - Validation Loss: 1.1892551353999548
Epoch 7/1000 - Training Loss: 1.2021798023155756
Epoch 7/1000 - Validation Loss: 1.1650667360850744
Epoch 8/1000 - Training Loss: 1.1826469153165817
Epoch 8/1000 - Validation Loss: 1.1441071374075753
Epoch 9/1000 - Training Loss: 1.1656023902552468
Epoch 9/1000 - Validation Loss: 1.124387868813106
Epoch 10/1000 - Training Loss: 1.1502561505351747
Epoch 10/1000 - Validation Loss: 1.107170752116612
Epo

...

Epoch 1/1000 - Training Loss: 3.0166276537458734e+18
Epoch 1/1000 - Validation Loss: 9.547970782411209e+17
Epoch 2/1000 - Training Loss: 1.9509700564814397e+18
Epoch 2/1000 - Validation Loss: 2.5226010849279063e+18
Epoch 3/1000 - Training Loss: 2.2543075702342382e+18
Epoch 3/1000 - Validation Loss: 1.3333023755550438e+18
Epoch 4/1000 - Training Loss: 1.8021328218330048e+18
Epoch 4/1000 - Validation Loss: 6.166886922166131e+17
Epoch 5/1000 - Training Loss: 2.1705161826053466e+18
Epoch 5/1000 - Validation Loss: 1.3398958031894328e+18
Epoch 6/1000 - Training Loss: 1.8744101218551258e+18
Epoch 6/1000 - Validation Loss: 6.952012834115795e+17
Epoch 7/1000 - Training Loss: 1.652169521001136e+18
Epoch 7/1000 - Validation Loss: 5.945719520129563e+17
Epoch 8/1000 - Training Loss: 1.816926409641614e+18
Epoch 8/1000 - Validation Loss: 1.1795032388175508e+18
Epoch 9/1000 - Training Loss: 1.9806694755468388e+18
Epoch 9/1000 - Validation Loss: 1.313113143045821e+18
Epoch 10/1000 - Training Loss: 1.44

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 1/1000 - Training Loss: 2.781031205999177e+18
Epoch 1/1000 - Validation Loss: 9.227327520118632e+17
Epoch 2/1000 - Training Loss: 2.0535024547563156e+18
Epoch 2/1000 - Validation Loss: 1.1592588442026452e+18
Epoch 3/1000 - Training Loss: 2.471496901160375e+18
Epoch 3/1000 - Validation Loss: 6.931527803527496e+17
Epoch 4/1000 - Training Loss: 1.6846566045427397e+18
Epoch 4/1000 - Validation Loss: 1.2006998595874511e+18
Epoch 5/1000 - Training Loss: 1.8077715481841032e+18
Epoch 5/1000 - Validation Loss: 1.0551769235794541e+18
Epoch 6/1000 - Training Loss: 1.639883783003664e+18
Epoch 6/1000 - Validation Loss: 1.6847118286186647e+18
Epoch 7/1000 - Training Loss: 1.6510504764756646e+18
Epoch 7/1000 - Validation Loss: 1.8483721709995013e+18
Epoch 8/1000 - Training Loss: 1.5654443234302922e+18
Epoch 8/1000 - Validation Loss: 6.568443294232084e+17
Epoch 9/1000 - Training Loss: 1.8730843942998694e+18
Epoch 9/1000 - Validation Loss: 1.2238850747580698e+18
Epoch 10/1000 - Training Loss: 1.8

/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pfs/data5/home/ma/ma_ma/ma_prajula/sense_polar/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

KeyboardInterrupt: 